In [1]:
# Import libraries

import csv
import re
import time
import os
import sys

from tqdm import tqdm
from tqdm.notebook import trange, tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from urllib.parse import urljoin

from threading import Thread

import pandas as pd

In [2]:
base_path = os.path.join(os.path.abspath(''), 'data')

#driver = webdriver.Chrome()
driver = webdriver.Firefox()
driver.get('https://www.lvz.de/suche/?query=Russland')

- Put in your Filter if you want now
- Open some articles first to get rid of unwanted popups

In [62]:
# Webscraping algorithm

iterations = 193

regex = r"^Kostenpflichtig"

not_readable = []

hits = 3840

df = pd.DataFrame(columns=['title','author','category', 'date', 'text'])
df.index.name = 'row_id'

pbar = tqdm(total=hits, position=0, leave=True)
for i in range(iterations):

    driver.switch_to.window(driver.window_handles[0])

    links = driver.find_elements(By.CSS_SELECTOR, 'div#resultdata > a')

    for j in range(len(links)):
        pbar.update(1)
    
        driver.switch_to.window(driver.window_handles[0])

        links[j].send_keys(Keys.CONTROL + Keys.RETURN)
        driver.switch_to.window(driver.window_handles[-1])

        try:      

            wait = WebDriverWait(driver, 5) 
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'header h2')))           

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            title = soup.select_one('article header > div > h2').text

            re_list = re.findall(regex, title)
            if len(re_list) > 0:
                not_readable.append(driver.current_url)
                driver.close()
                
                continue
            
            resort = str(driver.current_url).split('/')[3]

            date = soup.select_one('article header time').text
            
            text = soup.select('article > div > div > p')
            text = [t.text for t in text]
            text = " ".join(text)

            author = soup.select_one('article header a[rel="author"]')
            if author is not None:
                author = author.text
            else:
                author = 'NA'
        except:
            print("Error loading page: ", driver.current_url)
            driver.close()

            continue

        driver.close()

        df2 = pd.DataFrame({'title': title, 'author': author,'category':resort, 'date': date, 'text': text},index=[0])
        df2 = df2.astype(str)
        
        df = pd.concat([df, df2], ignore_index=True)
        
        time.sleep(0.75) # To not overload the server (hopefully)

    driver.switch_to.window(driver.window_handles[0])
    
    time.sleep(0.2)

    try:
        wait = WebDriverWait(driver, 5) 
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.prev_next_area > span > a.next_btn')))
        driver.find_element(By.CSS_SELECTOR, 'div.prev_next_area > span > a.next_btn').click()
    except:
        print("maybe skipping page: ", i)
        continue

    time.sleep(0.5)

pbar.close()

  0%|          | 0/2299 [00:00<?, ?it/s]

Error loading page:  https://www.lvz.de/politik/russischer-truppenaufmarsch-nato-besorgt-wie-selten-zuvor-LDY2P6AZHBEILLKRM2ROU6QDMA.html
Error loading page:  https://www.lvz.de/wirtschaft/millionen-verbraucher-betroffen-gas-und-strompreise-steigen-im-winter-zum-teil-erheblich-3TF2L64IFPTW2DBMN3YSHE2C5E.html
Error loading page:  https://www.lvz.de/politik/baerbock-und-maas-besorgt-um-russische-menschenrechtsorganisation-L2VA7DAI7JOT54KC6EVP5ARKP4.html
Error loading page:  https://www.lvz.de/politik/bundeswehr-uebergibt-luftraumueberwachung-ueber-baltikum-an-italien-2GAHOY6EUFBVXOTX6O2HJS7G3A.html
Error loading page:  about:blank
maybe skipping page:  114
maybe skipping page:  115


In [63]:
df

,title,author,category,date,text
0,Putins Neujahrsansprache: Lebens­standard der ...,NA,politik,"31.12.2021, 17:48 Uhr",Moskau. Russlands Präsident Wladimir Putin hat...
1,"""Super Abschluss"": Skispringer Geiger und Eise...",NA,sport,"31.12.2021, 16:10 Uhr",Garmisch-Partenkirchen. Fröhlich wünschten Kar...
2,Silvester 2021: Wie in Deutschland gefeiert we...,Jens Strube,panorama,"31.12.2021, 14:42 Uhr",Hannover. Auf Weihnachten folgt Silvester – un...
3,"Erst Diplomatie, dann Eishockey: Putin und Luk...",NA,politik,"31.12.2021, 11:03 Uhr",St. Petersburg. Am Rande von diplomatischen Ge...
4,Biden und Putin telefonieren zum Konflikt um d...,NA,politik,"31.12.2021, 10:20 Uhr",Wilmington. Zum zweiten Mal binnen weniger Woc...
...,...,...,...,...,...
1968,Friedensforscher: Biden macht die Welt nicht a...,NA,politik,"02.01.2021, 09:39 Uhr",Osnabrück. Mit dem neuen US-Präsidenten Joe Bi...
1969,"Krisen und Konflikte, die uns auch 2021 weiter...",NA,politik,"01.01.2021, 12:00 Uhr",Berlin. Nach dem Corona-Krisenjahr 2020 dürfte...
1970,Esa-Astronaut Maurer ist begeistert von geplan...,NA,wissen,"01.01.2021, 11:03 Uhr",Houston/Köln. Der Esa-Astronaut Matthias Maure...
1971,Das internationale Wahljahr 2021 – Rutte könnt...,NA,politik,"01.01.2021, 09:00 Uhr",Berlin. In Deutschland steht mit der Bundestag...


In [65]:
df.to_csv(f"G:\coding\PcToLaptop\CER\project\data\\lv_articles_20_0112_1.csv",quoting=csv.QUOTE_NONNUMERIC)

In [64]:
len(not_readable) + len(df)

2073